# Siglent Oscilloscope Control - Interactive Tutorial

This notebook demonstrates how to use the Siglent-Oscilloscope package to control your oscilloscope, capture waveforms, and perform measurements.

## Prerequisites

- Siglent-Oscilloscope package installed: `pip install Siglent-Oscilloscope`
- Oscilloscope connected to your network
- Oscilloscope IP address (check in Utility → I/O → LAN settings)

## 1. Setup and Connection

First, let's import the necessary modules and connect to the oscilloscope.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from siglent import Oscilloscope

# Configure matplotlib for better plots
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

In [ ]:
# Replace with your oscilloscope's IP address
SCOPE_IP = "192.168.1.100"  # Change this!

# Connect to oscilloscope
scope = Oscilloscope(SCOPE_IP, timeout=5.0)
scope.connect()

# Get device information
print("Connected to:", scope.identify())
print("\nDevice Info:")
for key, value in scope.device_info.items():
    print(f"  {key}: {value}")

## 2. Channel Configuration

Let's configure a channel for measurement.

In [ ]:
# Configure Channel 1
scope.channel1.enable()
scope.channel1.coupling = "DC"
scope.channel1.voltage_scale = 1.0  # 1V/div
scope.channel1.voltage_offset = 0.0
scope.channel1.probe_ratio = 10.0  # 10X probe

# Get current configuration
config = scope.channel1.get_configuration()
print("Channel 1 Configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")

## 3. Trigger Setup

Configure the trigger for stable waveform capture.

In [ ]:
# Set up edge trigger on Channel 1
scope.trigger.mode = "NORMAL"
scope.trigger.source = "C1"
scope.trigger.level = 0.0  # Trigger at 0V
scope.trigger.slope = "POS"  # Rising edge

print("Trigger configured:")
print(f"  Mode: {scope.trigger.mode}")
print(f"  Source: {scope.trigger.source}")
print(f"  Level: {scope.trigger.level}V")
print(f"  Slope: {scope.trigger.slope}")

## 4. Waveform Acquisition

Capture a waveform from the oscilloscope.

In [ ]:
# Start acquisition
scope.run()

# Capture waveform from Channel 1
waveform = scope.get_waveform(channel=1)

print(f"Waveform captured:")
print(f"  Samples: {len(waveform.time)}")
print(f"  Sample rate: {waveform.sample_rate/1e6:.1f} MSa/s")
print(f"  Time span: {waveform.time[-1]*1e3:.2f} ms")
print(f"  Voltage range: {waveform.voltage.min():.3f}V to {waveform.voltage.max():.3f}V")

## 5. Visualize the Waveform

Plot the captured waveform.

In [ ]:
# Plot waveform
plt.figure(figsize=(14, 6))
plt.plot(waveform.time * 1e3, waveform.voltage, 'b-', linewidth=1, label='Channel 1')
plt.grid(True, alpha=0.3)
plt.xlabel('Time (ms)')
plt.ylabel('Voltage (V)')
plt.title('Captured Waveform')
plt.legend()
plt.tight_layout()
plt.show()

# Statistics
print(f"\nWaveform Statistics:")
print(f"  Mean: {np.mean(waveform.voltage):.3f}V")
print(f"  RMS: {np.sqrt(np.mean(waveform.voltage**2)):.3f}V")
print(f"  Peak-to-Peak: {np.ptp(waveform.voltage):.3f}V")
print(f"  Min: {np.min(waveform.voltage):.3f}V")
print(f"  Max: {np.max(waveform.voltage):.3f}V")

## 6. Automated Measurements

Use the oscilloscope's built-in measurements.

In [ ]:
# Perform measurements
freq = scope.measurement.measure_frequency(1)
vpp = scope.measurement.measure_vpp(1)
vrms = scope.measurement.measure_rms(1)
period = scope.measurement.measure_period(1)
duty = scope.measurement.measure_duty_cycle(1)

print("Automated Measurements:")
print(f"  Frequency: {freq/1e3:.3f} kHz")
print(f"  Period: {period*1e6:.2f} μs")
print(f"  Vpp: {vpp:.3f} V")
print(f"  Vrms: {vrms:.3f} V")
print(f"  Duty Cycle: {duty:.1f}%")

## 7. FFT Analysis

Perform frequency domain analysis.

In [ ]:
from scipy import signal

# Compute FFT
window = signal.windows.hann(len(waveform.voltage))
fft = np.fft.fft(waveform.voltage * window)
freq_axis = np.fft.fftfreq(len(waveform.voltage), 1/waveform.sample_rate)

# Only positive frequencies
positive_freq = freq_axis[:len(freq_axis)//2]
magnitude_db = 20 * np.log10(np.abs(fft[:len(fft)//2]) + 1e-10)

# Plot FFT
plt.figure(figsize=(14, 6))
plt.plot(positive_freq/1e3, magnitude_db, 'r-', linewidth=1)
plt.grid(True, alpha=0.3)
plt.xlabel('Frequency (kHz)')
plt.ylabel('Magnitude (dB)')
plt.title('FFT Spectrum')
plt.xlim(0, waveform.sample_rate/2/1e3)
plt.tight_layout()
plt.show()

# Find peaks
peaks, _ = signal.find_peaks(magnitude_db, height=-20, distance=50)
if len(peaks) > 0:
    print(f"\nPeak Frequencies:")
    for i, peak in enumerate(peaks[:5]):  # Top 5 peaks
        print(f"  Peak {i+1}: {positive_freq[peak]/1e3:.2f} kHz ({magnitude_db[peak]:.1f} dB)")

## 8. Multiple Channel Capture

Capture and compare multiple channels.

In [ ]:
# Enable Channel 2
scope.channel2.enable()
scope.channel2.coupling = "DC"
scope.channel2.voltage_scale = 1.0

# Capture both channels
wf1 = scope.get_waveform(channel=1)
wf2 = scope.get_waveform(channel=2)

# Plot both
plt.figure(figsize=(14, 8))

plt.subplot(2, 1, 1)
plt.plot(wf1.time * 1e3, wf1.voltage, 'b-', linewidth=1, label='CH1')
plt.grid(True, alpha=0.3)
plt.ylabel('Voltage (V)')
plt.title('Channel 1')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(wf2.time * 1e3, wf2.voltage, 'r-', linewidth=1, label='CH2')
plt.grid(True, alpha=0.3)
plt.xlabel('Time (ms)')
plt.ylabel('Voltage (V)')
plt.title('Channel 2')
plt.legend()

plt.tight_layout()
plt.show()

## 9. Save Waveform Data

Export captured waveforms to files.

In [ ]:
# Save to different formats
scope.waveform.save_waveform(wf1, "waveform_ch1.csv", format="CSV")
scope.waveform.save_waveform(wf1, "waveform_ch1.npz", format="NPZ")

print("Waveform saved to:")
print("  waveform_ch1.csv")
print("  waveform_ch1.npz")

# Optional: Save to HDF5 (requires h5py)
try:
    scope.waveform.save_waveform(wf1, "waveform_ch1.h5", format="HDF5")
    print("  waveform_ch1.h5")
except ImportError:
    print("  (HDF5 format requires: pip install 'Siglent-Oscilloscope[hdf5]')")

## 10. Cleanup

Disconnect from the oscilloscope.

In [ ]:
# Disconnect
scope.disconnect()
print("Disconnected from oscilloscope")

## Next Steps

- Explore the [examples](https://github.com/little-did-I-know/Siglent-Oscilloscope/tree/main/examples) directory for more advanced use cases
- Check out the [automation API](https://github.com/little-did-I-know/Siglent-Oscilloscope#programmatic-data-collection--automation) for batch processing
- Try the GUI application: `siglent-gui`
- Read the full [documentation](https://github.com/little-did-I-know/Siglent-Oscilloscope#readme)

## Useful Resources

- [GitHub Repository](https://github.com/little-did-I-know/Siglent-Oscilloscope)
- [PyPI Package](https://pypi.org/project/Siglent-Oscilloscope/)
- [Report Issues](https://github.com/little-did-I-know/Siglent-Oscilloscope/issues)
- [Contributing Guide](https://github.com/little-did-I-know/Siglent-Oscilloscope/blob/main/CONTRIBUTING.md)